In [1]:

import pandas as pd
import numpy as np
import unicodedata
import re, os
import json


import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.stats import zscore

import pandas as pd
import time
import random
from requests import get
from bs4 import BeautifulSoup

import acquire
import GB_prepare

In [2]:
REPOS = acquire.scrape_github_data()

In [3]:
df_Repos = pd.DataFrame(REPOS)

In [4]:
df_Repos.head()

,repo,language,readme_contents
0,zmb3/spotify,Go,\nSpotify\n=======\n\n[![GoDoc](https://godoc....
1,spotDL/spotify-downloader,Python,<!--- mdformat-toc start --slug=github --->\n\...
2,Spotifyd/spotifyd,Rust,# Spotifyd <!-- omit in toc -->\n<!-- ALL-CONT...
3,Rigellute/spotify-tui,Rust,# Spotify TUI\n\n![Continuous Integration](htt...
4,JohnnyCrazy/SpotifyAPI-NET,C#,"\n<h1 align=""center"">\n <p align=""center"">Spo..."


In [5]:
GB_prepare.prepare_df(df_Repos,'readme_contents', extra_words = ['also', '&#9', 'e', 'f', 'ou', 'et', 'n', '1', "'", ';', '3', 'e', 'p'])

,repo,language,readme_contents,clean,stemmed,lemmatized,original_length,stem_length,lem_length,original_word_count,stemmed_word_count,lemmatized_word_count
0,zmb3/spotify,Go,\nSpotify\n=======\n\n[![GoDoc](https://godoc....,potify oochttpsgodocorggithubcomzmb3spotifysta...,potifi oochttpsgodocorggithubcomzmb3spotifysta...,potify oochttpsgodocorggithubcomzmb3spotifysta...,3918,2177,2466,503,257,257
1,spotDL/spotify-downloader,Python,<!--- mdformat-toc start --slug=github --->\n\...,mdformattoc start sluggithub indexmd docs div ...,mdformattoc start sluggithub indexmd doc div a...,mdformattoc start sluggithub indexmd doc div a...,4300,2616,2794,471,262,262
2,Spotifyd/spotifyd,Rust,# Spotifyd <!-- omit in toc -->\n<!-- ALL-CONT...,potifyd omit toc remove modify section ontribu...,potifyd omit toc remov modifi section ontribut...,potifyd omit toc remove modify section ontribu...,2338,1516,1614,235,138,138
3,Rigellute/spotify-tui,Rust,# Spotify TUI\n\n![Continuous Integration](htt...,potify ontinuous ntegrationhttpsgithubcomigell...,potifi ontinu ntegrationhttpsgithubcomigellute...,potify ontinuous ntegrationhttpsgithubcomigell...,59878,31783,32543,3857,2423,2343
4,JohnnyCrazy/SpotifyAPI-NET,C#,"\n<h1 align=""center"">\n <p align=""center"">Spo...",h1 aligncenter aligncenterpotifyp hrefhttpsjoh...,h1 aligncent aligncenterpotifyp hrefhttpsjohnn...,h1 aligncenter aligncenterpotifyp hrefhttpsjoh...,2782,1620,1746,272,158,159
...,...,...,...,...,...,...,...,...,...,...,...,...
995,root-host/Spotify-AdBlock,None,,,,,0,0,0,0,0,0
996,mattiasahlsen/spotify-queue,JavaScript,# spotify-app\n\n## deployed on\nhttps://colla...,spotifyapp deployed httpscollabqueuecom roject...,spotifyapp deploy httpscollabqueuecom roject s...,spotifyapp deployed httpscollabqueuecom roject...,355,208,240,48,27,27
997,anhjesslyn/spotify-account-creator,PHP,# Spotify Account Creator\n![license](https://...,potify ccount reator licensehttpsimgshieldsiog...,potifi ccount reator licensehttpsimgshieldsiog...,potify ccount reator licensehttpsimgshieldsiog...,809,546,567,73,46,46
998,possan/atari-spotify-app,Python,# Running Spotify on an 8-bit Atari\n\n> ⚠️ **...,unning potify 8bit tari isclaimer code hacky p...,un potifi 8bit tari isclaim code hacki provid ...,unning potify 8bit tari isclaimer code hacky p...,5984,3290,3610,959,470,470


In [6]:
df_Repos.head()

,repo,language,readme_contents,clean,stemmed,lemmatized,original_length,stem_length,lem_length,original_word_count,stemmed_word_count,lemmatized_word_count
0,zmb3/spotify,Go,\nSpotify\n=======\n\n[![GoDoc](https://godoc....,potify oochttpsgodocorggithubcomzmb3spotifysta...,potifi oochttpsgodocorggithubcomzmb3spotifysta...,potify oochttpsgodocorggithubcomzmb3spotifysta...,3918,2177,2466,503,257,257
1,spotDL/spotify-downloader,Python,<!--- mdformat-toc start --slug=github --->\n\...,mdformattoc start sluggithub indexmd docs div ...,mdformattoc start sluggithub indexmd doc div a...,mdformattoc start sluggithub indexmd doc div a...,4300,2616,2794,471,262,262
2,Spotifyd/spotifyd,Rust,# Spotifyd <!-- omit in toc -->\n<!-- ALL-CONT...,potifyd omit toc remove modify section ontribu...,potifyd omit toc remov modifi section ontribut...,potifyd omit toc remove modify section ontribu...,2338,1516,1614,235,138,138
3,Rigellute/spotify-tui,Rust,# Spotify TUI\n\n![Continuous Integration](htt...,potify ontinuous ntegrationhttpsgithubcomigell...,potifi ontinu ntegrationhttpsgithubcomigellute...,potify ontinuous ntegrationhttpsgithubcomigell...,59878,31783,32543,3857,2423,2343
4,JohnnyCrazy/SpotifyAPI-NET,C#,"\n<h1 align=""center"">\n <p align=""center"">Spo...",h1 aligncenter aligncenterpotifyp hrefhttpsjoh...,h1 aligncent aligncenterpotifyp hrefhttpsjohnn...,h1 aligncenter aligncenterpotifyp hrefhttpsjoh...,2782,1620,1746,272,158,159


In [7]:
labels = pd.concat([df_Repos.language.value_counts(),
                    df_Repos.language.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels.head()


,n,percent
JavaScript,244,0.253112
Python,197,0.204357
TypeScript,70,0.072614
Shell,37,0.038382
Java,36,0.037344


In [8]:
Java_words = (' '.join(df_Repos[df_Repos.language == 'JavaScript'].lemmatized)).split()
python_words = (' '.join(df_Repos[df_Repos.language == 'Python'].lemmatized)).split()
type_words= (' '.join(df_Repos[df_Repos.language == 'TypeScript'].lemmatized)).split()
Shell_words= (' '.join(df_Repos[df_Repos.language == 'Shell'].lemmatized)).split()
all_words = (' '.join(df_Repos.lemmatized)).split()


In [9]:
Java_freq = pd.Series(Java_words).value_counts()
python_freq = pd.Series(python_words).value_counts()
all_freq = pd.Series(all_words).value_counts()


Java_freq.head(20)

potify      1076
track        460
npm          412
use          395
playlist     383
run          367
file         364
app          360
user         299
install      257
need         251
code         250
build        249
project      247
token        226
using        222
j            209
data         203
eact         202
example      190
dtype: int64

In [10]:
type_freq = pd.Series(type_words).value_counts()

In [11]:
word_counts = (pd.concat([Java_freq, python_freq, all_freq, type_freq], axis=1, sort=True)
              .set_axis(['java','python', 'all', 'TypeScript'], axis=1, inplace=False)
              .fillna(0)
              .apply(lambda s: s.astype(int)))

In [12]:
word_counts.head(20)

,java,python,all,TypeScript
0,58,52,470,32
00,7,0,23,0
000,0,0,7,0
0000,0,1,42,1
000000,0,0,4,0
0000010,0,0,1,0
00005,0,0,1,0
00008000,0,1,1,0
0002,0,0,1,0
00059633501235151045timesignature,0,0,1,0


In [13]:
word_counts.sort_values(by='all', ascending=False).head(50)

,java,python,all,TypeScript
potify,1076,948,4764,367
track,460,344,1992,151
playlist,383,527,1791,183
use,395,271,1418,91
file,364,372,1352,65
song,163,418,1161,85
install,257,369,1126,71
user,299,148,1097,118
run,367,238,1059,83
app,360,108,1058,80
